# V1 - with only documents

In [1]:
#!pip install python-Levenshtein
import pandas as pd
import Levenshtein as ln
import networkx as nx
from ast import literal_eval

from tqdm import tqdm
import pickle
#G = nx.DiGraph()

In [2]:
ids_question_map = {}
question_ids_map = {}
nline = 0

with open("../data/qg.txt", "r") as f:
    with tqdm(total=1563052) as progress_bar:
        while line := f.readline():
            ids_question_map[nline] = line.strip('\n').lower()
            question_ids_map[line.strip('\n').lower()] = nline
            nline += 1
            progress_bar.update(1)

pickle.dump(ids_question_map, open("../index/ids_question_map.pkl", "wb"))
pickle.dump(question_ids_map, open("../index/question_ids_map.pkl", "wb"))

100%|█████████████████████| 1563052/1563052 [00:01<00:00, 836118.76it/s]


In [3]:
ids_question_map = pickle.load(open("../index/ids_question_map.pkl", "rb"))
question_ids_map = pickle.load(open("../index/question_ids_map.pkl", "rb"))

G = nx.DiGraph()

track_leafs = {}

with open("../data/qg.txt", "r") as f:
    nlines = 0 
    with tqdm(total=1563052) as progress_bar:
        while line := f.readline():
            if line!='':
                question = line.strip("\n").lower()
                tokenized = question.split()

                if not tokenized:
                    continue

                prev_node_entry = tokenized[0]
                for i in range(2, len(tokenized)+1):
                    if len(prev_node_entry.split()) == 1:
                        if prev_node_entry not in G:
                            G.add_node(prev_node_entry, starts=True)
                        else:
                            G.nodes[prev_node_entry]['starts'] = True
                    else:
                        if prev_node_entry not in G:
                            G.add_node(prev_node_entry, starts=False)
                        else:
                            G.nodes[prev_node_entry]['starts'] = False

                    #if " ".join(tokenized[:i]) not in G:
                    if len(" ".join(tokenized[:i]).split()) == 1:
                        if " ".join(tokenized[:i]) not in G:
                            G.add_node(" ".join(tokenized[:i]), starts=True)
                        else:
                            G.nodes[" ".join(tokenized[:i])]['starts'] = True
                    else:
                        if " ".join(tokenized[:i]) not in G:
                            G.add_node(" ".join(tokenized[:i]), starts=False)
                        else:
                            G.nodes[" ".join(tokenized[:i])]['starts'] = False

                    #G.nodes[prev_node_entry]['leaf'] = G.nodes[prev_node_entry].get('leaf', []) + [question_ids_map[question]]
                    #print(prev_node_entry, question)
                    a = track_leafs.get(prev_node_entry, [])
                    a.append(question_ids_map[question])
                    track_leafs[prev_node_entry] = a
                    G.add_edge(prev_node_entry, " ".join(tokenized[:i]))
                    prev_node_entry =  " ".join(tokenized[:i])
                a = track_leafs.get(question, [])
                a.append(question_ids_map[question])
                track_leafs[question] = a
                    
            progress_bar.update(1)

roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]



100%|█████████████████████▉| 1562895/1563052 [00:56<00:00, 27455.25it/s]


In [4]:
nx.write_gpickle(G, "../index/query_graph.gpickle")
pickle.dump(track_leafs, open("../index/track_leafs.pkl", "wb"))
# G = nx.read_gpickle("../index/query_graph.gpickle")
# track_leafs = pickle.load(open("../index/track_leafs.pkl", "rb"))
# roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

# ids_question_map = pickle.load(open("../index/ids_question_map.pkl", "rb"))
# question_ids_map = pickle.load(open("../index/question_ids_map.pkl", "rb"))

In [9]:
# recursion
import timeit


#query = "hwat ia the bst way to file".lower()
#query = "becoming a"
query = "do parents".lower()
#query = "hwat ia the".lower()
#query = "which is the difference between bile".lower()
#query = "what was the name given to a".lower()
#query = "bbilical defi".lower()
tokenized_query = query.split()
len_tokenized_query = len(tokenized_query)
node_list = []
def recurssive_search(comparison_term, G, idx=1, root=False, node_list=[], forked=False):
    new_term = []
    for node in (roots_list if root else G[" ".join(comparison_term.split()[:-1])]):
        
        # don't auto-correct on last-word
        if ((ln.distance(comparison_term, node) <= 2) if idx!=len_tokenized_query-1 else tokenized_query[-1] in node):
            if idx!=len_tokenized_query-1:
                node_list.extend(recurssive_search(node + " " + tokenized_query[idx+1], G, idx+1, root=False, node_list=node_list, forked=True))
            else:
                if node in G and node in track_leafs:
                    new_term.append(node)

              
    if forked:
        if idx==len_tokenized_query-1:
            return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
            
        return ""
    else:
        if len(tokenized_query)==1:
            if idx==len_tokenized_query-1:
                return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
        return node_list

start_time = timeit.default_timer()
res = recurssive_search(tokenized_query[0], G, idx=0, root=True)
print("Total time taken: ", timeit.default_timer()-start_time)

Total time taken:  0.018978670999786118


In [9]:
ln.distance("machine", "mcine")

2

In [10]:
res

['how parents talk to teenagers on sexual health issues',
 'how parents influence their children to start smoking',
 'how parents determine their own development',
 'how parents should be held responsible',
 'how parents should not be accountable for their children behavior',
 'how grandparents help',
 'how grandparents can help grandchildren',
 'how grandparents help one another',
 'how grandparents care for grandchildren in different ways',
 'how grandparents benefit from the adoption act',
 'how grandparents are benefiting from the foster care and adoption act',
 'how grandparents care for their grandchildren',
 'do grandparents really care what their grandchildren do',
 'do grandparents help each other',
 'do grandparents really love to read to grandchildren',
 'do grandparents help grandchildren',
 'do grandparents love their grandchildren for the love they give them',
 'do parents influence baby eye color',
 'do parents know what to do when their kid has a fever',
 'do parents ne

# V2 - documents & user queries

In [1]:
import pandas as pd
import Levenshtein as ln
import networkx as nx
from ast import literal_eval

from tqdm import tqdm
import pickle

In [2]:
# let's load v1 data

G = nx.read_gpickle("../index/query_graph.gpickle")
track_leafs = pickle.load(open("../index/track_leafs.pkl", "rb"))
roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

ids_question_map = pickle.load(open("../index/ids_question_map.pkl", "rb"))
question_ids_map = pickle.load(open("../index/question_ids_map.pkl", "rb"))

In [3]:
import csv
import sys
csv.field_size_limit(sys.maxsize)

old_lines = len(ids_question_map)
lines = len(ids_question_map)
with open("../data/queries.train.tsv", "r") as f:
    tsv_reader = csv.reader(f, delimiter="\t")
    for row in tsv_reader:
        ids_question_map[lines] = row[-1].lower()
        question_ids_map[row[-1].lower()] = lines
        lines += 1
lines - old_lines

808731

In [8]:
question_ids_map['do parents go to their kids college orientation']

1563277

In [4]:

with open("../data/queries.train.tsv", "r") as f:
    tsv_reader = csv.reader(f, delimiter="\t")
    nlines = 0 
    for row in tqdm(tsv_reader, total=808731):
        nlines+=1
        line = row[-1]
        if line!='':
            question = line.strip("\n").lower()
            tokenized = question.split()

            if not tokenized:
                continue
            
            prev_node_entry = tokenized[0]
            for i in range(2, len(tokenized)+1):
                if len(prev_node_entry.split()) == 1:
                    if prev_node_entry not in G:
                        G.add_node(prev_node_entry, starts=True)
                    else:
                        G.nodes[prev_node_entry]['starts'] = True
                else:
                    if prev_node_entry not in G:
                        G.add_node(prev_node_entry, starts=False)
                    else:
                        G.nodes[prev_node_entry]['starts'] = False

                #if " ".join(tokenized[:i]) not in G:
                if len(" ".join(tokenized[:i]).split()) == 1:
                    if " ".join(tokenized[:i]) not in G:
                        G.add_node(" ".join(tokenized[:i]), starts=True)
                    else:
                        G.nodes[" ".join(tokenized[:i])]['starts'] = True
                else:
                    if " ".join(tokenized[:i]) not in G:
                        G.add_node(" ".join(tokenized[:i]), starts=False)
                    else:
                        G.nodes[" ".join(tokenized[:i])]['starts'] = False

                #G.nodes[prev_node_entry]['leaf'] = G.nodes[prev_node_entry].get('leaf', []) + [question_ids_map[question]]
                #print(prev_node_entry, question)
                a = track_leafs.get(prev_node_entry, [])
                a.append(question_ids_map[question])
                track_leafs[prev_node_entry] = a
                G.add_edge(prev_node_entry, " ".join(tokenized[:i]))
                prev_node_entry =  " ".join(tokenized[:i])
            a = track_leafs.get(question, [])
            a.append(question_ids_map[question])
            track_leafs[question] = a

roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

100%|████████████████████████| 808731/808731 [00:30<00:00, 26348.01it/s]


In [2]:
# nx.write_gpickle(G, "../index/v2/query_graph.gpickle")
# pickle.dump(track_leafs, open("../index/v2/track_leafs.pkl", "wb"))
# pickle.dump(ids_question_map, open("../index/v2/ids_question_map.pkl", "wb"))
# pickle.dump(question_ids_map, open("../index/v2/question_ids_map.pkl", "wb"))

G = nx.read_gpickle("../index/v2/query_graph.gpickle")
track_leafs = pickle.load(open("../index/v2/track_leafs.pkl", "rb"))
ids_question_map = pickle.load(open("../index/v2/ids_question_map.pkl", "rb"))
question_ids_map = pickle.load(open("../index/v2/question_ids_map.pkl", "rb"))

roots_list = [i for i,j in G.nodes(data="starts", default=1) if j==True]

In [5]:
# recursion
import timeit


#query = "do parents".lower()
query = "what is presentation".lower()
#query = "becoming a".lower() # for v1 v2 comparison
#query = "hwat ia the".lower()
#query = "what was the name given to a".lower()
#query = "bbilical defn".lower()
#query = "what".lower()
tokenized_query = query.split()
len_tokenized_query = len(tokenized_query)
node_list = []
def recurssive_search(comparison_term, G, idx=1, root=False, node_list=[], forked=False):
    new_term = []
    for node in (roots_list if root else G[" ".join(comparison_term.split()[:-1])]):
        
        # don't auto-correct on last-word
        if ((ln.distance(comparison_term, node) <= 2) if idx!=len_tokenized_query-1 else tokenized_query[-1] in node):
            if idx!=len_tokenized_query-1:
                node_list.extend(recurssive_search(node + " " + tokenized_query[idx+1], G, idx+1, root=False, node_list=node_list, forked=True))
            else:
                if node in G and node in track_leafs:
                    new_term.append(node)

              
    if forked:
        if idx==len_tokenized_query-1:
            return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
            
        return ""
    else:
        if len(tokenized_query)==1:
            if idx==len_tokenized_query-1:
                return [ids_question_map[child] for term in new_term for child in track_leafs[term]]
        return node_list

start_time = timeit.default_timer()
res = recurssive_search(tokenized_query[0], G, idx=0, root=True)
print("Total time taken: ", timeit.default_timer()-start_time)

Total time taken:  0.038735168000130216


In [6]:
res

['what is representation meaning',
 'what is representation means',
 'what is representation yahoo answers',
 'what is representation in legal terms',
 'what is representation mean',
 'what is representation in the legal dictionary',
 'what is representation means synonyms',
 'what is representation in government',
 'what is representation in law',
 'what is representation',
 'what is representation in english',
 'what is representation in house of representatives based on population',
 'what is representation in ece',
 'what is representation',
 'what is representation fees geneve',
 'what is representations',
 'what is presentation gi sample density',
 'what is representational democracy',
 'when is misrepresentation puffery']

In [28]:
[ids_question_map[child] for child in track_leafs['do parents']]

['do parents influence baby eye color',
 'do parents know what to do when their kid has a fever',
 'do parents need to have a child for family leave',
 'do parents go to their kids college orientation',
 "do parents affect children's self esteem",
 'do parents of groom give them a gift',
 'do parents count as immediate family']